In [2]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

import pandas as pd
import numpy as np
import sys
import h5py
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
from keras.layers.advanced_activations import PReLU
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
import matplotlib.patches as mpatches

from keras.layers import Dense, Activation, concatenate, Reshape,Concatenate
from keras.layers.embeddings import Embedding
from keras import regularizers
from keras.initializers import he_normal
from keras.regularizers import l2
from keras.activations import selu
from keras.optimizers import Adam



%matplotlib inline

/home/coder.chenshicheng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
ntrain = len(train)
ids = test['id']
test['loss'] = np.nan
dataset = pd.DataFrame()
joined = pd.concat([train, test])
f_num = [f for f in train.columns if 'cont' in f]
f_cat = [f for f in train.columns if 'cat' in f]
for column in f_cat:
    if (train[column].nunique() != test[column].nunique()
        or (train[column].unique() != test[column].unique()).any()):
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        
        remove = ((set_train - set_test)|(set_test - set_train))
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x
        myfilter = lambda x: np.nan if x in remove else x
        joined[column] = joined[column].apply(filter_cat, 1)

    dataset[column] = pd.factorize(joined[column].values, sort=True)[0] + 1

train = joined[joined['loss'].notnull()]
test = joined[joined['loss'].isnull()]

shift = 200
#train_y = np.log(train['loss'] + shift)
train_y=train['loss']
train_x = train.drop(['loss', 'id'], 1)
test = test.drop(['loss', 'id'], 1)

scaler = StandardScaler()
contx = scaler.fit_transform(train_x[f_num])
catx = pd.get_dummies(data=train_x[f_cat],columns=f_cat,dummy_na=False)
test = pd.get_dummies(data=test,columns=f_cat)

In [4]:
test.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,...,cat116_MU,cat116_MV,cat116_MW,cat116_O,cat116_Q,cat116_R,cat116_S,cat116_T,cat116_U,cat116_Y
0,0.321594,0.299102,0.246911,0.402922,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,...,0,0,0,0,0,0,0,0,0,0
1,0.634734,0.620805,0.654310,0.946616,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,...,0,0,0,0,0,0,0,0,0,0
2,0.290813,0.737068,0.711159,0.412789,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,...,0,0,0,0,0,0,0,0,0,0
3,0.268622,0.681761,0.592681,0.354893,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,...,0,0,0,0,0,0,0,0,0,0
4,0.553846,0.299102,0.263570,0.696873,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,...,0,0,0,0,0,0,0,0,0,0


In [5]:
for i in catx.columns:
    a = catx[i].sum()
    b = test[i].sum()
    if(a < 100 | b < 100):
        del catx[i]
        del test[i]

In [6]:
print (catx.shape,contx.shape)
test = test.values
train_x = np.concatenate((contx,catx.values),axis=1)

print (train_x.shape)
print (test.shape)
print (train_y.shape)

(188318, 1065) (188318, 14)
(188318, 1079)
(125546, 1079)
(188318,)


In [7]:
import keras.backend as K

def mae_score(y_true, y_pred):
    return K.mean(K.abs(K.exp(y_true)-K.exp(y_pred)), axis=-1)

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', 
                             verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [8]:
# http://cs231n.github.io/neural-networks-3/

import numpy as np
import scipy.sparse as sp

from keras import backend as K
from keras.callbacks import Callback
from keras.models import load_model

import sys
import warnings
class ExponentialMovingAverage(Callback):
    """create a copy of trainable weights which gets updated at every
       batch using exponential weight decay. The moving average weights along
       with the other states of original model(except original model trainable
       weights) will be saved at every epoch if save_mv_ave_model is True.
       If both save_mv_ave_model and save_best_only are True, the latest
       best moving average model according to the quantity monitored
       will not be overwritten. Of course, save_best_only can be True
       only if there is a validation set.
       This is equivalent to save_best_only mode of ModelCheckpoint
       callback with similar code. custom_objects is a dictionary
       holding name and Class implementation for custom layers.
       At end of every batch, the update is as follows:
       mv_weight -= (1 - decay) * (mv_weight - weight)
       where weight and mv_weight is the ordinal model weight and the moving
       averaged weight respectively. At the end of the training, the moving
       averaged weights are transferred to the original model.
       """
    def __init__(self, decay=0.999, filepath='model2/model{epoch:02d}-{val_loss:.2f}.hdf5',
                 save_mv_ave_model=False, verbose=0,
                 save_best_only=False, monitor='val_loss', mode='auto',
                 save_weights_only=False, custom_objects={}):
        self.decay = decay
        self.filepath = filepath
        self.verbose = verbose
        self.save_mv_ave_model = save_mv_ave_model
        self.save_weights_only = save_weights_only
        self.save_best_only = save_best_only
        self.monitor = monitor
        self.custom_objects = custom_objects  # dictionary of custom layers
        self.sym_trainable_weights = None  # trainable weights of model
        self.mv_trainable_weights_vals = None  # moving averaged values
        self.epochs = 0

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor:
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_train_begin(self, logs={}):
        self.sym_trainable_weights = self.model.trainable_weights
        # Initialize moving averaged weights using original model values
        self.mv_trainable_weights_vals = {x.name: K.get_value(x).copy() for x in
                                          self.sym_trainable_weights}
        if self.verbose:
            print('Created a copy of model weights to initialize moving averaged weights.')
    def on_train_end(self, logs={}):
        '''old = K.get_value(self.model.trainable_weights[0])
        name = self.model.trainable_weights[0].name
        assert((old != self.mv_trainable_weights_vals[name]).any())'''
        
        for weight in self.sym_trainable_weights:
            K.set_value(weight, self.mv_trainable_weights_vals[weight.name])

        '''old = K.get_value(self.model.trainable_weights[0])
        name = self.model.trainable_weights[0].name
        assert((old == self.mv_trainable_weights_vals[name]).all())'''
            
    def on_batch_end(self, batch, logs={}):
        if(self.epochs + 5 > self.params['epochs']):
            for weight in self.sym_trainable_weights:
                old_val = self.mv_trainable_weights_vals[weight.name].copy()
                self.mv_trainable_weights_vals[weight.name] = \
                    old_val * self.decay + (1.0 - self.decay) * K.get_value(weight).copy()
            #assert((old_val == self.mv_trainable_weights_vals[weight.name]).all())
            #assert((old_val == K.get_value(weight)).all())

    def on_epoch_end(self, epoch, logs={}):
        self.epochs += 1
        if(self.epochs + 5 > self.params['epochs']):
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            self.model.save(filepath, overwrite=True)
        

In [9]:
dataset.head()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116
0,1,2,1,2,1,1,1,1,2,1,...,10,7,39,27,3,20,54,1,15,239
1,1,2,1,1,1,1,1,1,2,2,...,11,11,30,59,1,23,37,1,15,74
2,1,2,1,1,2,1,1,1,2,2,...,6,1,3,79,1,29,5,1,9,135
3,2,2,1,2,1,1,1,1,2,1,...,11,11,30,61,3,40,4,1,15,68
4,1,2,1,2,1,1,1,1,2,2,...,7,2,57,45,3,51,37,1,11,44


In [10]:
datasetX = dataset.iloc[:ntrain]

In [11]:
datasetX.shape

(188318, 116)

In [12]:
contx.shape

(188318, 14)

In [13]:
X_list = []

X_list.append(contx)
for i in f_cat:
    X_list.append(datasetX[i].values)

In [14]:
from keras.layers import Flatten, Merge
from keras.layers import Concatenate

In [17]:
def hyper_model(seed = None):
    models = []

    cur = Sequential()
    cur.add(Dense(13, input_dim=14,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.01)))
    cur.add(BatchNormalization())
    models.append(cur)
    input_shape = 7
    for i in f_cat:
        cur = Sequential()
        num = dataset[i].nunique()
        cur.add(Embedding(num, num // 2, input_length=1))
        input_shape += num // 2
        cur.add(Flatten())
        models.append(cur)
    
    
    model = Sequential()
    model.add(Merge(models, mode='concat'))
    '''model.add(Dense(437, kernel_initializer=he_normal(seed = seed)
                    ,kernel_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.536))'''
    
    model.add(Dense(300, kernel_initializer='glorot_normal',kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.4))
    
    model.add(Dense(73, kernel_initializer='glorot_normal',kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.233))
    
    model.add(Dense(1, kernel_initializer='glorot_normal',kernel_regularizer=l2(0.01)))
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #model.compile(optimizer='adadelta',loss = 'mae',metrics = [mae_score])
    model.compile(optimizer=adam,loss = 'mae')
    return model

In [18]:
f = hyper_model(seed = 0)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
fit = f.fit(X_list, train_y, validation_split=0.2, batch_size=256,
              epochs=30, verbose=1)#, callbacks=[ExponentialMovingAverage()])
pred =f.predict(X_list, batch_size=1024)
#score = mean_absolute_error(np.exp(yte), np.exp(pred))
score = mean_absolute_error(train_y, pred)

/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 150654 samples, validate on 37664 samples
Epoch 1/30
150654/150654 [==============================] - 40s 267us/step - loss: 3014.4508 - val_loss: 2951.7708
Epoch 2/30
150654/150654 [==============================] - 36s 239us/step - loss: 2877.5393 - val_loss: 2771.1379
Epoch 3/30
150654/150654 [==============================] - 36s 237us/step - loss: 2614.5349 - val_loss: 2472.4463
Epoch 4/30
150654/150654 [==============================] - 35s 235us/step - loss: 2286.0198 - val_loss: 2072.3309
Epoch 5/30
150654/150654 [==============================] - 36s 237us/step - loss: 2085.5115 - val_loss: 2069.2152
Epoch 6/30
150654/150654 [==============================] - 36s 236us/step - loss: 1923.5672 - val_loss: 1971.4767
Epoch 7/30
150654/150654 [==============================] - 35s 235us/step - loss: 1824.0327 - val_loss: 1869.5227
Epoch 8/30
150654/150654 [==============================] - 36s 239us/step - loss: 1772.5982 - val_loss: 1739.4178
Epoch 9/30
150654/150654 [===

In [ ]:
nnmodel={}
nnmodel.clear()
def cross_validate_mlp(mlp_func, nfolds=10,nbags=5):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nbags,))
    stack_train = np.zeros((nbags,len(train_y)))
    stack_test = np.zeros((nbags,len(test)))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        for bag in range(nbags):
            nnmodel['nn%d',k*10+bag*1] = mlp_func(seed = k*10+bag*1)
            early_stopping = EarlyStopping(monitor='val_loss', patience=5)
            fit = nnmodel['nn%d',k*10+bag*1].fit(xtr, ytr, validation_split=0.2, batch_size=128,
                          epochs=30, verbose=1, callbacks=[ExponentialMovingAverage()])
            pred = nnmodel['nn%d',k*10+bag*1].predict(xte, batch_size=1024)
            #score = mean_absolute_error(np.exp(yte), np.exp(pred))
            score = mean_absolute_error(yte, pred)
            val_scores[bag] += score
            #stack_train[bag][test_index] = pred[:,0]
            print ("nfold:{},bag:{}".format(k,bag),score)
    for bag in range(nbags):
        val_scores[bag] = val_scores[bag] / float(nfolds)
    
    
    return val_scores

cv_score = cross_validate_mlp(hyper_model)
print ("CV score for the final model:", cv_score)

Though this model is not adapted for mere 30 epochs of training, nor for 3-fold CV (I used 5-fold on Kaggle), even though this is a single unbagged model which has been cross-validated on three folds only, we see a very good score:
`CV = 1150` (your score may vary a little).

By the way, this single model, bagged, 5-fold CVed, scored 1116.28 on Kaggle LB.

As we see, this model is considerably better than any other models we had so far. We now take it as the second part of our final ensemble.

In [ ]:
'l3-nn': {
        'predictions': l2_predictions,
        'n_bags': 4,
        'model': Keras(nn_lr, lambda: {'l1': 1e-5, 'l2': 1e-5, 'n_epoch': 30, 'batch_size': 128, 'optimizer': SGD(3e-2, momentum=0.8, nesterov=True, decay=3e-5), 'callbacks': [ExponentialMovingAverage(save_mv_ave_model=False)]}),
    },
     'optimizer': SGD(1e-4, momentum=0.9, nesterov=True, decay=5e-5)
            'optimizer': SGD(1e-5, momentum=0.9, nesterov=True, decay=5e-5)

In [ ]:
{'hidden1_dropout': 0.5368421052631579, 'hidden1_units': 437, 'hidden2_dropout': 0.40000000000000002, 
 'hidden2_units': 182, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 73, 'optimizer': 'adadelta',
 'wdecay': 0.0020300000000000001}

In [251]:
def cross_validate_mlp(mlp_func, nfolds=3):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        fit = mlp.fit(xtr, ytr, validation_split=0.1, batch_size=128, 
                      epochs=10, verbose=0, callbacks=[early_stopping])
        pred = mlp.predict(xte, batch_size=512)
        score = mean_absolute_error(np.exp(yte), np.exp(pred))
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [252]:
# VERSION 4. Insights:
# – why not to test 4-layer architectures?
# — we need to introduce new optimizers
# — adding batch normalization (https://arxiv.org/abs/1502.03167)

# Describing the search space
space = {'hidden1_dropout': hp.choice('hidden1_dropout', np.linspace(0.4,0.6,20)),
        'hidden2_dropout': hp.choice('hidden2_dropout', np.linspace(0.2,0.5,10)),
        'hidden3_dropout': hp.choice('hidden3_dropout', np.linspace(0.1,0.5,10)),
         'hidden1_units': hp.choice('hidden1_units', np.linspace(300,550,30,dtype='int32')),
         'hidden2_units': hp.choice('hidden2_units', np.linspace(100,300,30,dtype='int32')),
         'hidden3_units': hp.choice('hidden3_units', np.linspace(20,80,30,dtype='int32')),
         'optimizer': hp.choice('optimizer', ['adadelta','adam']),
         'wdecay':hp.choice('wdecay', np.linspace(0.0001,0.01,1000)),
        }

# Implementing a function to minimize
def hyperopt_search(params):
    print ('Model Testing:', params)
    def mlp_model():
        model = Sequential()
        model.add(Dense(params['hidden1_units'], input_dim=train_x.shape[1], kernel_initializer='he_normal',
                        kernel_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden1_dropout']))
        
        model.add(Dense(params['hidden2_units'], kernel_initializer='he_normal',
                        kernel_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden2_dropout']))

        model.add(Dense(params['hidden3_units'], kernel_initializer='he_normal',
                        kernel_regularizer=l2(params['wdecay']))) 
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden3_dropout']))
        
        model.add(Dense(1, kernel_initializer='he_normal',kernel_regularizer=l2(params['wdecay'])))
        model.compile(loss=mae_score,metrics=[mae_score], optimizer=params['optimizer'])
        return model
    
    cv_score = cross_validate_mlp(mlp_model)
    return {'loss': cv_score, 'status': STATUS_OK}


trials = Trials()

# UNCOMMENT THE NEXT LINE TO LAUNCH HYPEROPT:
best = fmin(hyperopt_search, space, algo=tpe.suggest, max_evals = 100, trials=trials)

Model Testing: {'hidden1_dropout': 0.59999999999999998, 'hidden1_units': 463, 'hidden2_dropout': 0.46666666666666667, 'hidden2_units': 155, 'hidden3_dropout': 0.18888888888888888, 'hidden3_units': 20, 'optimizer': 'adam', 'wdecay': 0.0012891891891891893}
Fold 0, MAE: 1664.3154518123333
Fold 1, MAE: 1789.6478768193072
Fold 2, MAE: 2230.6679431902985
3-fold CV score: 1894.877090607313
Model Testing: {'hidden1_dropout': 0.50526315789473686, 'hidden1_units': 308, 'hidden2_dropout': 0.20000000000000001, 'hidden2_units': 244, 'hidden3_dropout': 0.41111111111111109, 'hidden3_units': 28, 'optimizer': 'adadelta', 'wdecay': 0.0075522522522522527}
Fold 0, MAE: 2663.589775993532
Fold 1, MAE: 2550.3793884253055
Fold 2, MAE: 2698.309167392798
3-fold CV score: 2637.426110603878
Model Testing: {'hidden1_dropout': 0.59999999999999998, 'hidden1_units': 394, 'hidden2_dropout': 0.23333333333333334, 'hidden2_units': 120, 'hidden3_dropout': 0.18888888888888888, 'hidden3_units': 65, 'optimizer': 'adam', 'wde

KeyboardInterrupt: 